In [1]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import pandas as pd
import numpy as np
from quantfreedom.class_practice.enums import *
from quantfreedom.class_practice.base import backtest_df_only, plot_one_result
from quantfreedom.indicators.talib_ind import from_talib



price_data = pd.read_hdf("../../tests/data/prices.hd5")
%load_ext autoreload
%autoreload 2


In [3]:
price_data

symbol                BTCUSDT                               EOSUSD            \
candle_info              open      high       low     close   open high  low   
open_time                                                                      
2020-10-01 00:00:00 10,759.00 10,817.50 10,759.00 10,790.00   2.58 2.58 2.58   
2020-10-01 00:30:00 10,790.00 10,790.50 10,775.00 10,777.00   2.58 2.58 2.58   
2020-10-01 01:00:00 10,777.00 10,793.00 10,777.00 10,793.00   2.58 2.59 2.58   
2020-10-01 01:30:00 10,793.00 10,836.00 10,792.50 10,822.00   2.59 2.59 2.59   
2020-10-01 02:00:00 10,822.00 10,841.50 10,808.00 10,808.50   2.59 2.60 2.59   
...                       ...       ...       ...       ...    ...  ...  ...   
2021-04-14 21:30:00 62,703.50 62,812.50 62,615.50 62,795.00   7.58 7.62 7.54   
2021-04-14 22:00:00 62,795.00 62,825.00 62,591.00 62,680.00   7.61 7.73 7.55   
2021-04-14 22:30:00 62,680.00 62,993.00 62,680.00 62,849.50   7.65 7.77 7.65   
2021-04-14 23:00:00 62,849.50 62,999.00 62,750.00 62,999.00   7.71 7.79 7.67   
2021-04-14 23:30:00 62,999.00 62,999.00 62,606.00 62,921.00   7.78 7.78 7.63   

symbol                      ETHUSD                            LTCUSDT         \
candle_info         close     open     high      low    close    open   high   
open_time                                                                      
2020-10-01 00:00:00  2.58   359.85   361.45   358.85   358.95     NaN    NaN   
2020-10-01 00:30:00  2.58   358.95   359.75   358.95   359.25     NaN    NaN   
2020-10-01 01:00:00  2.59   359.25   359.80   358.80   359.75     NaN    NaN   
2020-10-01 01:30:00  2.59   359.75   361.40   359.75   361.20     NaN    NaN   
2020-10-01 02:00:00  2.59   361.20   364.05   361.20   362.95     NaN    NaN   
...                   ...      ...      ...      ...      ...     ...    ...   
2021-04-14 21:30:00  7.61 2,406.20 2,440.00 2,397.20 2,434.05  269.26 281.37   
2021-04-14 22:00:00  7.65 2,434.05 2,445.00 2,418.65 2,427.00  280.82 282.02   
2021-04-14 22:30:00  7.71 2,427.00 2,438.60 2,423.50 2,431.30  278.49 281.31   
2021-04-14 23:00:00  7.78 2,431.30 2,450.00 2,431.30 2,445.95  277.50 280.17   
2021-04-14 23:30:00  7.68 2,445.95 2,445.95 2,417.40 2,438.20  279.05 280.95   

symbol                            XRPUSD                  
candle_info            low  close   open high  low close  
open_time                                                 
2020-10-01 00:00:00    NaN    NaN   0.24 0.24 0.24  0.24  
2020-10-01 00:30:00    NaN    NaN   0.24 0.24 0.24  0.24  
2020-10-01 01:00:00    NaN    NaN   0.24 0.24 0.24  0.24  
2020-10-01 01:30:00    NaN    NaN   0.24 0.24 0.24  0.24  
2020-10-01 02:00:00    NaN    NaN   0.24 0.24 0.24  0.24  
...                    ...    ...    ...  ...  ...   ...  
2021-04-14 21:30:00 269.11 280.82   1.78 1.82 1.77  1.82  
2021-04-14 22:00:00 278.24 278.49   1.82 1.82 1.77  1.78  
2021-04-14 22:30:00 276.64 277.50   1.78 1.82 1.78  1.80  
2021-04-14 23:00:00 277.46 279.05   1.80 1.84 1.79  1.83  
2021-04-14 23:30:00 277.95 279.41   1.83 1.85 1.82  1.84  

[9408 rows x 20 columns]

In [4]:
rsi_ind = from_talib(
    func_name='rsi',
    nickname='rsi',
    price_data=price_data,
    input_names = ['close'],
    parameters = {'timeperiod': [15, 20]}
)
entries = rsi_ind.is_below(
    user_args=[40, 50],
    candle_ohlc="close",
)

In [25]:
account_state = AccountState()
backtest_settings = BacktestSettings(
    divide_records_array_size_by=2,
    gains_pct_filter=10.0,
    total_trade_filter=10,
    upside_filter=0.0,
)
exchange_settings = ExchangeSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01, .02]) / 100,
    sl_based_on_lookback=np.array([50]),
    risk_reward=np.array([2.0, 5.0]),
    leverage_type=np.array([LeverageType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitType.RiskReward]),
    max_equity_risk_pct=np.array([3]) / 100,
    order_type=np.array([OrderType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]) / 100,
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([0.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
)

In [43]:
order_records_df, strat_res_df, order_set_res_df = backtest_df_only(
    account_state=account_state,
    order_settings_arrays=order_settings_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=exchange_settings,
    price_data=price_data,
    entries=entries,
)

Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 5
Total indicator settings per symbol: 4
Total indicator settings to test: 20
Total order settings per symbol: 4
Total order settings to test: 20
Total combinations of settings to test: 400

Total candles per symbol: 9,408
Total candles to test: 752,640


In [44]:
strat_res_df

,symbol_idx,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
0,1,2,2,67.00,17.04,89.55,0.94,170.36,"1,170.36"
1,1,2,3,67.00,17.04,89.55,0.94,170.36,"1,170.36"
2,1,2,0,67.00,13.72,89.55,0.94,137.22,"1,137.21"
3,1,2,1,67.00,13.72,89.55,0.94,137.22,"1,137.21"
4,1,0,2,67.00,12.77,88.06,0.86,127.74,"1,127.74"
5,1,0,3,67.00,12.77,88.06,0.86,127.74,"1,127.74"
6,1,0,0,67.00,11.22,88.06,0.85,112.24,"1,112.24"
7,1,0,1,67.00,11.22,88.06,0.85,112.24,"1,112.24"
8,3,1,1,396.00,14.00,51.26,0.34,140.04,"1,140.04"
9,3,1,0,397.00,13.76,51.13,0.30,137.64,"1,137.64"


In [73]:
order_set_res_df

,symbol_idx,or_set_idx,increase_position_type,leverage_type,max_equity_risk_pct,order_type,risk_account_pct_size,risk_reward,sl_based_on_add_pct,sl_based_on_lookback,...,sl_to_be_based_on_candle_body_type,sl_to_be_when_pct_from_candle_body,sl_to_be_zero_or_entry_type,static_leverage,stop_loss_type,take_profit_type,tp_fee_type,trail_sl_based_on_candle_body_type,trail_sl_by_pct,trail_sl_when_pct_from_candle_body
0,1,0,4,2,0.03,1,0.01,2.00,0.00,50,...,0,0.00,0,0.00,1,1,1,2,0.01,0.03
1,1,1,4,2,0.03,1,0.01,2.00,0.00,50,...,0,0.00,0,0.00,1,1,1,2,0.01,0.03
2,1,2,4,2,0.03,1,0.01,5.00,0.00,50,...,0,0.00,0,0.00,1,1,1,2,0.01,0.03
3,1,3,4,2,0.03,1,0.01,5.00,0.00,50,...,0,0.00,0,0.00,1,1,1,2,0.01,0.03
4,1,0,4,2,0.03,1,0.01,2.00,0.00,50,...,0,0.00,0,0.00,1,1,1,2,0.01,0.03
5,1,1,4,2,0.03,1,0.01,2.00,0.00,50,...,0,0.00,0,0.00,1,1,1,2,0.01,0.03
6,1,2,4,2,0.03,1,0.01,5.00,0.00,50,...,0,0.00,0,0.00,1,1,1,2,0.01,0.03
7,1,3,4,2,0.03,1,0.01,5.00,0.00,50,...,0,0.00,0,0.00,1,1,1,2,0.01,0.03
8,3,0,4,2,0.03,1,0.01,2.00,0.00,50,...,0,0.00,0,0.00,1,1,1,2,0.01,0.03
9,3,1,4,2,0.03,1,0.01,2.00,0.00,50,...,0,0.00,0,0.00,1,1,1,2,0.01,0.03


In [46]:
order_records_df

,symbol_idx,ind_set_idx,or_set_idx,bar_idx,equity,available_balance,cash_borrowed,cash_used,average_entry,fees_paid,...,possible_loss,entry_size,entry_price,exit_price,position_size,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,1,0,0,32,"1,000.00",989.62,34.43,10.38,2.57,NaN,...,10.00,44.81,2.57,NaN,44.81,NaN,22.21,2.00,44.68,3.72
1,1,0,0,270,"1,000.00",989.62,34.43,10.38,2.57,NaN,...,10.00,44.81,2.57,NaN,44.81,NaN,-7.28,2.76,44.68,3.72
2,1,0,0,271,"1,000.00",989.62,34.43,10.38,2.57,NaN,...,10.00,44.81,2.57,NaN,44.81,NaN,-10.40,2.84,44.68,3.72
3,1,0,0,272,"1,004.60","1,004.60",NaN,NaN,NaN,0.06,...,NaN,NaN,NaN,2.84,NaN,4.60,NaN,NaN,NaN,NaN
4,1,0,0,351,"1,004.60",994.05,36.01,10.56,2.55,NaN,...,10.05,46.56,2.55,NaN,46.56,NaN,21.48,2.00,43.22,3.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11851,3,3,3,9393,"1,150.32","1,133.27",525.14,17.05,260.21,NaN,...,11.50,542.19,260.21,NaN,542.19,NaN,-3.59,269.56,10.65,287.92
11852,3,3,3,9394,"1,169.13","1,169.13",NaN,NaN,NaN,0.66,...,NaN,NaN,NaN,269.56,NaN,18.81,NaN,NaN,NaN,NaN
11853,3,3,3,9396,"1,169.13","1,154.01",322.62,15.12,263.82,NaN,...,11.69,337.75,263.82,NaN,337.75,NaN,3.34,255.00,17.35,309.59
11854,3,3,3,9403,"1,169.13","1,154.01",322.62,15.12,263.82,NaN,...,11.69,337.75,263.82,NaN,337.75,NaN,-5.59,278.56,17.35,309.59


In [75]:
order_settings_arrays

array([2])

In [89]:
test_list = [1,2,3]
if 9 not in test_list:
    print('yes')
    test_list.append(9)
test_list

yes


[1, 2, 3, 9]

In [94]:
np.array([[1,2,3], [4,5,6]])[:,[1,2]]

array([[2, 3],
       [5, 6]])